In [28]:
import numpy as np
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda
from tensorflow.python.keras import utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

In [29]:
pip install gensim


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\adity\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [30]:
import gensim

In [31]:
# Open a text file in write mode
with open('corona.txt', 'w') as file:
    # Write the message to the file
    file.write("""
The speed of transmission is an important point of difference between the two viruses.
Influenza has a shorter median incubation period (the time from infection to appearance of symptoms)
and a shorter serial interval (the time between successive cases) than COVID-19 virus.
The serial interval for COVID-19 virus is estimated to be 5-6 days, while for influenza virus, the serial interval is 3 days.
This means that influenza can spread faster than COVID-19.

Further, transmission in the first 3-5 days of illness, or potentially pre-symptomatic transmission –
transmission of the virus before the appearance of symptoms – is a major driver of transmission for influenza.
In contrast, while we are learning that there are people who can shed COVID-19 virus 24-48 hours prior to symptom onset,
at present, this does not appear to be a major driver of transmission.

The reproductive number – the number of secondary infections generated from one infected individual –
is understood to be between 2 and 2.5 for COVID-19 virus, higher than for influenza.
However, estimates for both COVID-19 and influenza viruses are very context and time-specific, making direct comparisons more difficult.
""")

In [35]:
data=open('corona.txt','r')
corona_data = [text for text in data if text.count(' ') >= 2]
vectorize = Tokenizer()
corona_data

['The speed of transmission is an important point of difference between the two viruses.\n',
 'Influenza has a shorter median incubation period (the time from infection to appearance of symptoms)\n',
 'and a shorter serial interval (the time between successive cases) than COVID-19 virus.\n',
 'The serial interval for COVID-19 virus is estimated to be 5-6 days, while for influenza virus, the serial interval is 3 days.\n',
 'This means that influenza can spread faster than COVID-19.\n',
 'Further, transmission in the first 3-5 days of illness, or potentially pre-symptomatic transmission –\n',
 'transmission of the virus before the appearance of symptoms – is a major driver of transmission for influenza.\n',
 'In contrast, while we are learning that there are people who can shed COVID-19 virus 24-48 hours prior to symptom onset,\n',
 'at present, this does not appear to be a major driver of transmission.\n',
 'The reproductive number – the number of secondary infections generated from one

In [33]:
vectorize.fit_on_texts(corona_data)
corona_data = vectorize.texts_to_sequences(corona_data)

[[1, 38, 2, 3, 9, 39, 40, 41, 2, 42, 14, 1, 43, 23], [4, 44, 11, 24, 45, 46, 47, 1, 15, 25, 48, 10, 26, 2, 27], [12, 11, 24, 16, 17, 1, 15, 14, 49, 50, 18, 5, 6, 7], [1, 16, 17, 8, 5, 6, 7, 9, 51, 10, 19, 20, 52, 21, 28, 8, 4, 7, 1, 16, 17, 9, 29, 21], [30, 53, 31, 4, 32, 54, 55, 18, 5, 6], [56, 3, 33, 1, 57, 29, 20, 21, 2, 58, 59, 60, 61, 62, 3, 13], [3, 2, 1, 7, 63, 1, 26, 2, 27, 13, 9, 11, 34, 35, 2, 3, 8, 4], [33, 64, 28, 65, 22, 66, 31, 67, 22, 68, 69, 32, 70, 5, 6, 7, 71, 72, 73, 74, 10, 75, 76], [77, 78, 30, 79, 80, 81, 10, 19, 11, 34, 35, 2, 3], [1, 82, 36, 13, 1, 36, 2, 83, 84, 85, 25, 86, 87, 88, 13], [9, 89, 10, 19, 14, 37, 12, 37, 20, 8, 5, 6, 7, 90, 18, 8, 4], [91, 92, 8, 93, 5, 6, 12, 4, 23, 22, 94, 95, 12, 15, 96, 97, 98, 99, 100, 101]]


In [18]:
# Find total no of words and total no of sentences.
total_vocab = sum(len(s) for s in corona_data)
word_count = len(vectorize.word_index) + 1
window_size = 2

In [19]:
# Generate the pairs of Context words and target words
def cbow_model(data, window_size, total_vocab):
    total_length = window_size*2
    for text in data:
        text_len = len(text)
        for idx, word in enumerate(text):
            context_word = []
            target   = []
            begin = idx - window_size
            end = idx + window_size + 1
            context_word.append([text[i] for i in range(begin, end) if 0 <= i < text_len and i != idx])
            target.append(word)
            contextual = sequence.pad_sequences(context_word, total_length=total_length)
            final_target = utils.to_categorical(target, total_vocab)
            yield(contextual, final_target)

In [20]:
model = Sequential()
model.add(Embedding(input_dim=total_vocab, output_dim=100, input_length=window_size*2))
model.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(100,)))
model.add(Dense(total_vocab, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
for i in range(10):
    cost = 0
    for x, y in cbow_model(data, window_size, total_vocab):
        cost += model.train_on_batch(x,y)
    print(i, cost)

0 0
1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 0


In [21]:
# Create vector file of some word for testing
dimensions=100
vect_file = open('vectors.txt' ,'w')
vect_file.write('{} {}\n'.format(total_vocab,dimensions))

8

In [22]:
# Assign weights to your trained model
weights = model.get_weights()[0]
for text, i in vectorize.word_index.items():
    final_vec = ' '.join(map(str, list(weights[i, :])))
    vect_file.write('{} {}\n'.format(text, final_vec))
vect_file.close()

In [23]:
# Use the vectors created in Gemsim
cbow_output = gensim.models.KeyedVectors.load_word2vec_format('vectors.txt', binary = False, limit=100,encoding='cp1252')

In [25]:
cbow_output.most_similar(positive=['virus'])

[('3', 0.20359352231025696),
 ('the', 0.19478558003902435),
 ('speed', 0.17708979547023773),
 ('individual', 0.15886743366718292),
 ('means', 0.15616390109062195),
 ('onset', 0.14859910309314728),
 ('infection', 0.14327740669250488),
 ('pre', 0.14294208586215973),
 ('who', 0.13404597342014313),
 ('48', 0.12396558374166489)]